-------------------------
#### Information value
------------------------

In [1]:
# Importing libraries
import numpy as np
import pandas as pd

In [33]:
location = r'D:\AI-DATASETS\01-MISC\bank-full.csv'

In [34]:
# load the training data from iris.txt
df = pd.read_csv(location , delimiter = ";")

In [35]:
df.shape

(45211, 17)

In [36]:
df.sample(5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
42782,33,technician,married,tertiary,no,1699,no,no,cellular,28,jan,202,1,83,3,success,yes
24809,35,technician,single,tertiary,no,172,yes,yes,cellular,18,nov,52,1,-1,0,unknown,no
15418,25,management,single,tertiary,no,0,no,yes,cellular,18,jul,342,1,-1,0,unknown,no
44904,48,management,single,tertiary,no,334,yes,no,cellular,28,sep,600,2,92,12,success,yes
33420,29,technician,single,tertiary,no,2269,yes,no,cellular,20,apr,53,1,346,1,failure,no


In [41]:
df_dropped = df.drop(['age', 'balance', 'day', 'duration', 'pdays', 'campaign',	'previous'], axis=1).copy()

In [42]:
df_dropped.sample(5)

,job,marital,education,default,housing,loan,contact,month,poutcome,y
2608,blue-collar,married,primary,no,no,no,unknown,may,unknown,no
42898,services,single,tertiary,no,no,no,cellular,feb,success,no
11484,admin.,married,secondary,no,no,no,unknown,jun,unknown,no
42657,blue-collar,married,secondary,no,no,no,cellular,jan,other,yes
19325,management,divorced,secondary,no,no,no,cellular,aug,unknown,no


In [46]:
df_dropped.dtypes

job          object
marital      object
education    object
default      object
housing      object
loan         object
contact      object
month        object
poutcome     object
y            object
dtype: object

In [47]:
# Selecting only the categorical columns
categorical_columns = df.select_dtypes(include=['object']).columns

# Displaying unique categories for each categorical variable
for col in categorical_columns:
    unique_categories = df[col].unique()
    print(f"{col}: {unique_categories}")

job: ['management' 'technician' 'entrepreneur' 'blue-collar' 'unknown'
 'retired' 'admin.' 'services' 'self-employed' 'unemployed' 'housemaid'
 'student']
marital: ['married' 'single' 'divorced']
education: ['tertiary' 'secondary' 'unknown' 'primary']
default: ['no' 'yes']
housing: ['yes' 'no']
loan: ['no' 'yes']
contact: ['unknown' 'cellular' 'telephone']
month: ['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'jan' 'feb' 'mar' 'apr' 'sep']
poutcome: ['unknown' 'failure' 'other' 'success']
y: ['no' 'yes']


In [48]:
def calculate_iv(df, categorical_var, target_var):
    # Calculate the percentage of events and non-events for each category
    event_rates = df.groupby(categorical_var)[target_var].mean()
    non_event_rates = 1 - event_rates
    
    # Avoid division by zero when calculating odds ratio
    non_event_rates[non_event_rates == 0] = 0.0001
    
    # Calculate odds ratio and log odds for each category
    odds_ratio = event_rates / non_event_rates
    log_odds = np.log(odds_ratio)
    
    # Calculate IV for each category
    iv_values = (event_rates - non_event_rates) * log_odds
    
    # Sum up IV values across categories to get overall IV
    overall_iv = iv_values.sum()
    
    # Display IV values for each category and overall IV
    iv_df = pd.DataFrame({categorical_var: event_rates.index, 'IV': iv_values})
    
    return iv_df, overall_iv


In [52]:
# Assuming your target variable is 'y' and it's binary ('yes' or 'no')
# Convert the target variable to 0 or 1
df_dropped['y'] = df_dropped['y'].map({'no': 0, 'yes': 1})

In [53]:
# Assuming 'df' is your DataFrame with multiple categorical variables and binary target 'target'
categorical_vars = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
target_var = 'y'

In [56]:
# Calculate IV for each categorical variable
overall_iv_values = []

for cat_var in categorical_vars:
    iv_df, overall_iv = calculate_iv(df_dropped, cat_var, target_var)
    #print(f"\nInformation Value for {cat_var}:\n{iv_df}")
    print(f"\nOverall IV for {cat_var}: {overall_iv}")
    overall_iv_values.append({'Variable': cat_var, 'Overall IV': overall_iv})



Overall IV for job: 17.762299129136586

Overall IV for marital: 4.480613726504907

Overall IV for education: 6.200872093228684

Overall IV for default: 3.880493166530274

Overall IV for housing: 3.171421724978705

Overall IV for loan: 3.727151844844264

Overall IV for contact: 5.487951094845211

Overall IV for month: 12.978299420125127

Overall IV for poutcome: 4.571692866645868


**Interpretation of IV:**

- The higher the IV, the more predictive power the variable is assumed to have. Commonly, an IV value:
    - Less than 0.02: Weak predictor
    - 0.02 to 0.1: Medium predictor
    - Greater than 0.5: Strong predictor